### Check input files



In [8]:
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('../datasets'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


../datasets/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
../datasets/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv


In [18]:
def readDrop(path):
    data = pd.read_csv(path)
    data = data.drop(['Timestamp'], axis=1)
    data = data.drop_duplicates()
    return data

### Import train data and create DataFrame object

In [9]:
# rawsData = pd.read_csv('../datasets/firstDataset.csv')

ftpSshBruteForce = pd.read_csv(
    '../datasets/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv').drop_duplicates()
# dosHttpHulk = pd.read_csv(
#     '../datasets/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv')
# ddosHttpUdp = pd.read_csv(
#     '../datasets/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv')
# ddosLoicHoic = pd.read_csv(
#     '../datasets/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv')
# webXSSBruteForceSql = pd.read_csv(
#     '../datasets/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv')
# webXSSBruteForceSql_2 = pd.read_csv(
#     '../datasets/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv')
# infiltration = pd.read_csv(
#     '../datasets/Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv')
# infiltration_2 = pd.read_csv(
#     '../datasets/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv')
# bot = pd.read_csv(
#     '../datasets/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv')

print("The files have been imported")


The files have been imported


In [19]:
dosGoldeyeSlowloris = readDrop(
    '../datasets/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv')

In [20]:
dosGoldeyeSlowloris.shape[0]

884658

In [10]:
ftpSshBruteForce.shape[0]

822947

In [ ]:
rawsData = pd.concat([ftpSshBruteForce, dosGoldeyeSlowloris, dosHttpHulk, ddosHttpUdp, ddosLoicHoic,
                       webXSSBruteForceSql, webXSSBruteForceSql_2, infiltration, infiltration_2, bot], axis=0).drop_duplicates()

In [ ]:
rawsData = rawsData.drop(['Timestamp'], axis=1)


In [ ]:
len(rawsData)


In [ ]:
rawsData.keys()


In [ ]:
LABEL = 'Label'
NORMAL = 'Benign'
FTP = 'FTP-BruteForce'
SSH = 'SSH-Bruteforce'


In [ ]:
# rawNormal = rawsData[rawsData[LABEL] == NORMAL]
# rawFTP = rawsData[rawsData[LABEL] == FTP]
# rawSSH = rawsData[rawsData[LABEL] == SSH]


In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
# import plotly as py
# import plotly.express as px
init_notebook_mode(connected=True)


def showAttackDistribution(data):
    counted = data.value_counts()

    fig = go.Figure(data=[
        go.Bar(name=NORMAL,
               y=[counted[NORMAL]],
               x=[NORMAL],
               text=str(counted[NORMAL]),
               orientation='v',
               textposition='outside',),
        go.Bar(name=FTP,
               y=[counted[FTP]],
               x=[FTP],
               text=str(counted[FTP]),
               orientation='v',
               textposition='outside',),
        go.Bar(name=SSH,
               y=[counted[SSH]],
               x=[SSH],
               text=str(counted[SSH]),
               orientation='v',
               textposition='outside',)
    ])
    # Change the bar mode
    fig.update_layout(
        width=800,
        height=600,
        title=f'Labels Distribution',
        yaxis_title='Number of attacks',
        xaxis_title='Attack Name',)
    iplot(fig)


In [ ]:
showAttackDistribution(rawsData['Label'])


In [ ]:
def getSliceFromRawData():
    return rawsData[['Protocol',
                     'Fwd Pkt Len Max',
                     'Fwd Pkt Len Max',
                     'Bwd Pkt Len Mean',
                     'Fwd Pkt Len Std',
                     'Bwd Pkt Len Max',
                     'Bwd Pkt Len Min',
                     'Bwd Pkt Len Std',
                     'Bwd IAT Tot',
                     'Bwd IAT Mean',
                     'Bwd IAT Max',
                     'Pkt Len Min',
                     'Pkt Len Max',
                     'PSH Flag Cnt',
                     'Fwd Header Len',
                     'SYN Flag Cnt',
                     'RST Flag Cnt',
                     'ACK Flag Cnt',
                     'URG Flag Cnt',
                     'ECE Flag Cnt',
                     'Fwd Seg Size Avg',
                     'Init Fwd Win Byts',
                     'Fwd Act Data Pkts',
                     'Fwd Seg Size Min',
                     'Down/Up Ratio',
                     'Bwd Pkts/s',
                     'Fwd Pkts/s',
                     'Pkt Size Avg',
                     'Idle Mean',
                     'Label']].copy()


In [ ]:
selectedData = getSliceFromRawData()
allDatasetToTest = getSliceFromRawData()

# selectedData = rawsData.copy()
# allDatasetToTest = rawsData.copy()


In [ ]:
withoutDuplicates = selectedData.drop_duplicates().copy()
withoutDuplicates.info()

In [ ]:
showAttackDistribution(withoutDuplicates[LABEL])


In [ ]:
def shortData(data, featureName):
    shortedNormal = data[data[featureName] ==
                         NORMAL].sample(100, ignore_index=True, random_state=32)
    shortedFTP = data[data[featureName] == FTP].sample(10, ignore_index=True, random_state=32)
    shortedSSH = data[data[featureName] == SSH].sample(100, ignore_index=True, random_state=32)
    return pd.concat([shortedNormal, shortedFTP, shortedSSH], axis=0)


In [ ]:
toTrainModel = shortData(withoutDuplicates, LABEL).copy()
showAttackDistribution(toTrainModel[LABEL])


In [ ]:

def makeOnlyAttackOrNot(dataToTransform):
    columnToTransform = [LABEL]
    for colKey in columnToTransform:
        dataToTransform[colKey] = dataToTransform[colKey].map(
            lambda i: 0 if i == NORMAL else 1)
        # dataToTransform[colKey] = LabelEncoder().fit_transform(dataToTransform[colKey])

    return dataToTransform


In [ ]:
toTrainModel = makeOnlyAttackOrNot(toTrainModel)
allDatasetToTest = makeOnlyAttackOrNot(allDatasetToTest)

In [ ]:
def removeNanInf(data):
    maxNonInf = data.max().loc[lambda value: value < np.Inf].max()
    data = data.replace(np.nan, 0)
    data = data.replace(np.inf, maxNonInf)
    return data

In [ ]:
toTrainModel = removeNanInf(toTrainModel)
allDatasetToTest = removeNanInf(allDatasetToTest)

In [ ]:
# from sklearn.preprocessing import StandardScaler


# def scale(dataToTransform, columntoTransform):

#     ss_dict = {col: StandardScaler() for col in columntoTransform}

#     for colKey in columntoTransform:
#         dataToTransform[colKey] = ss_dict[colKey].fit_transform(
#             np.array(dataToTransform[colKey]).reshape(-1, 1))

#     return dataToTransform


In [ ]:
# numericColumntoTransform = withoutDuplicates.keys()
# withoutDuplicates = scale(withoutDuplicates, numericColumntoTransform)


### Heatmap
#### Heatmap provide us information on how much the individual features are interpedent. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

f, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(abs(toTrainModel.corr()), annot=True,
            linewidths=0.5, fmt='.2f', ax=ax)


Save label

In [ ]:
labels = toTrainModel[LABEL]
toTrainModel = toTrainModel.drop([LABEL], axis=1)


In [ ]:
def printNanFinite(data):
    print('Is any Nan:', np.any(np.isnan(data)))
    print('Is all finite:', np.all(np.isfinite(data)))


In [ ]:
printNanFinite(toTrainModel)
print('\n')
printNanFinite(allDatasetToTest)


Scale numeric values

#### After preprocess

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    toTrainModel, labels, test_size=0.33, random_state=32)


X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')


In [ ]:
isRFC = True
isMLPC = False
isSequential = False


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense


def model():
    if isRFC:
        model = RandomForestClassifier(
            n_estimators=100, max_depth=5, min_samples_leaf=4)
    elif isMLPC:
        model = MLPClassifier(random_state=32)
    else:
        model = Sequential([
            Dense(256, activation='softplus', kernel_initializer='glorot_uniform',
                  input_dim=X_train.shape[1]),
            Dense(128, activation='softplus',
                  kernel_initializer='glorot_uniform'),
            Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid')
        ])

        model.compile(loss='binary_crossentropy',
                      optimizer='Adam')

    return model


In [ ]:
model = model()


In [ ]:
model.fit(X_train, y_train)


In [ ]:

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def confMatrix(yTrain, xTrainPreds, yTest, xTestPreds):
    # Your code here
    cmTrain = confusion_matrix(yTrain, xTrainPreds)
    X_train_disp = ConfusionMatrixDisplay(confusion_matrix=cmTrain)

    cmTest = confusion_matrix(yTest, xTestPreds)
    X_test_disp = ConfusionMatrixDisplay(confusion_matrix=cmTest)
    # End

    _, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=150)

    X_train_disp.plot(ax=ax[0])
    ax[0].set_title("Train")

    X_test_disp.plot(ax=ax[1])
    ax[1].set_title("Test")
    plt.show()


In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

def printScores(name, labels, predicts):
    print(name )
    print('Accuracy: %.3f ' % accuracy_score(labels, predicts))
    print('Precision: %.3f ' % precision_score(labels, predicts))
    print('Recall: %.3f' % recall_score(labels, predicts))
    print('F1 Score: %.3f' % f1_score(labels, predicts))
    print('\n')

In [ ]:


X_train_preds = model.predict(X_train)
X_test_preds = model.predict(X_test)

if isSequential:
    print('Train auc:', roc_auc_score(y_train, X_train_preds))
    print('Test auc:', roc_auc_score(y_test, X_test_preds))
else:
    printScores('Train:', y_train, X_train_preds)
    printScores('Test:', y_test, X_test_preds)


In [ ]:
confMatrix(y_train, X_train_preds, y_test, X_test_preds)


In [ ]:
allDatasetLabels = allDatasetToTest[LABEL]
allDatasetToTest = allDatasetToTest.drop([LABEL], axis=1)

allDatasetLabels.to_csv('allDatasetLabels.csv')
allDatasetToTest.to_csv('allDatasetToTest.csv')


In [ ]:


AllDatasetPreds = model.predict(allDatasetToTest)

if isSequential:
    print('Auc:', roc_auc_score(allDatasetLabels, AllDatasetPreds))
else:
    printScores('All dataset:', allDatasetLabels, AllDatasetPreds)


In [ ]:
allDatasetConfusionMatrix = confusion_matrix(allDatasetLabels, AllDatasetPreds)
allDataSetMatrixDisplay = ConfusionMatrixDisplay(confusion_matrix=allDatasetConfusionMatrix)
_, ax = plt.subplots(figsize=(8, 4), dpi=130)
ax.set_title("All dataset")
allDataSetMatrixDisplay.plot(ax=ax)
plt.show()
